In [4]:
!pip install np_utils

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Input, LSTM, concatenate, AveragePooling2D
from tensorflow.keras.layers import Subtract, Conv2D, GlobalMaxPooling2D, Reshape, Conv2DTranspose, UpSampling2D, Embedding
from tensorflow.keras import regularizers
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from matplotlib import pyplot as plt

import numpy as np

In [0]:
#parameters
epsilon = 0.01  #the quantity of perturbation we add to the images
step_pgd = 20   #the number of steps taken during the PGD algo
eta = 0.005     #used in PGD algo


#parameters

epsilon_l2 = 0.4  #the quantity of perturbation we add to the images norm L2
epsilon_inf = 0.031 #norm infini

step_pgd = 20   #the number of steps taken during the PGD algo
eta_l2 = 0.4     #used in PGD algo
eta_inf = 0.031

In [3]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow is already loaded. Please restart the runtime to change versions.


In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
x_train = x_train / 255
x_test = x_test / 255

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [8]:
# (Conv + ReLU + MaxPool)x3 + AvgPool + FC + ReLU + FC (fully connected)
input = tf.keras.Input(shape=(32, 32, 3))
x = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(input)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_last')(x) 

x = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(input)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_last')(x)

x = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(input)
x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_last')(x)

x = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(10, activation='softmax')(x)


adversarial = keras.models.Model(inputs=input, outputs=x)
adversarial.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
history = adversarial.fit(x_train, y_train, batch_size=64, epochs=20, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 43s 1ms/sample - loss: 1.6966 - acc: 0.3971 - val_loss: 1.4576 - val_acc: 0.4838
Epoch 2/20
40000/40000 [==============================] - 42s 1ms/sample - loss: 1.3664 - acc: 0.5187 - val_loss: 1.3113 - val_acc: 0.5454
Epoch 3/20
40000/40000 [==============================] - 42s 1ms/sample - loss: 1.2597 - acc: 0.5597 - val_loss: 1.2527 - val_acc: 0.5675
Epoch 4/20
40000/40000 [==============================] - 42s 1ms/sample - loss: 1.1942 - acc: 0.5874 - val_loss: 1.2290 - val_acc: 0.5706
Epoch 5/20
40000/40000 [==============================] - 41s 1ms/sample - loss: 1.1418 - acc: 0.6079 - val_loss: 1.1793 - val_acc: 0.5953
Epoch 6/20
40000/40000 [==============================] - 41s 1ms/sample - loss: 1.0993 - acc: 0.6223 - val_loss: 1.1532 - val_acc: 0.6076
Epoch 7/20
40000/40000 [==============================] - 41s 1ms/sample - loss: 1.0624 - acc: 0.6371 - val_loss: 1.

In [0]:
def NES_grad(sigma, n_sample = x_train.shape[0],n=[32,32,3]):
  g = np.zeros([n_sample,32,32,3])
  for i in range(n_sample):
    u = np.random.normal(0,1,n)
    #print(g[i][0])
    g[i] = g[i] + np.argmax(adversarial.predict(np.reshape(x_train[i] + sigma*u,[1,32,32,3]) , batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False),axis=1)*u
    #print(g[i][0])
    #print(adversarial.predict(np.reshape(x_train[i] + sigma*u,[1,32,32,3]), batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False))
    #print(adversarial.predict(np.reshape(x_train[i] - sigma*u,[1,32,32,3]), batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False))
    #g[i] = g[i] - np.argmax(adversarial.predict(np.reshape(x_train[i] - sigma*u,[1,32,32,3]), batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False),axis=1)*u
    #print(g[i][0])
    if (i%10==0):
      print(i)
  return(1/(2*n_sample*sigma)*g)
    

In [52]:
label = np.argmax(adversarial.predict(x_train[:2], batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False),axis =1 )
label

array([6, 9])

In [53]:
aa = NES_grad(7,1)

0


In [54]:
NES_grad(7,100)

0
10
20
30
40
50
60
70
80
90


array([[[[-2.62319735e-04,  4.02913112e-04, -1.31811285e-03],
         [ 3.28952518e-04,  2.99744307e-04,  8.57774999e-05],
         [-7.53109007e-05,  2.64009449e-04,  4.30041104e-04],
         ...,
         [-2.23522250e-04,  2.36682767e-05, -9.96833350e-04],
         [-5.07266139e-04, -4.13343741e-04, -1.92395376e-04],
         [ 2.64821195e-04,  1.08876013e-03,  4.21936377e-04]],

        [[ 6.62974254e-04,  3.99635903e-04,  3.18441188e-04],
         [ 3.59651866e-04,  2.90726356e-04,  4.22316216e-04],
         [-1.21979495e-03, -3.85149246e-04,  1.97314114e-04],
         ...,
         [-6.63963063e-04, -3.47426616e-05, -1.46950446e-04],
         [ 2.09716344e-04, -1.25715335e-03,  4.85859846e-04],
         [ 1.03045214e-03, -1.31918058e-03, -3.02516017e-04]],

        [[-8.15663977e-04, -6.06397734e-04, -6.18505386e-04],
         [ 2.79265474e-04,  1.63975616e-04, -2.17847744e-04],
         [ 1.13981035e-03, -3.69079210e-04, -6.99120586e-05],
         ...,
         [-4.64317048e-0